# Extract, Transform, Load (ETL)


As described in the Green Cloud Model, we will begin with the **ASK** and **GET** step. For **ASK**, we want to determine a forecasting model to project the housing market, which can be analyzed on a national, state, or county level. In order to do so, we will complete the **GET** step and use the Redfin Housing Market Data, which is thoroughly discussed and cited in `ReadMe_Data` in the MySQL folder for this project. A MySQL database was created with this data, where the necessary sql files can be found in the `MySQL` folder for this project, and a series of basic queries were written to get a better sense of the data. The results from the queries are stored in the table `Query_results`. This covers most of the necessary ETL we need. 

Next, we will connect to the database in Jupyter and look at any basic information related to the data set to check for null values and confirm that the data can be used for Exploratory Data Analysis (EDA) and building the forecasting model. 

In [1]:
import pymysql
import pandas as pd

In [2]:
pd.options.display.max_colwidth = 500

In [3]:
conn = pymysql.connect(
    host= 'MySQL_1',
    port=3306,
    user="Sarah",
    passwd='password',
    db="db",
    charset='utf8mb4')

In [4]:
curs = conn.cursor()

In [5]:
curs.execute("Show tables;")
 
result = pd.DataFrame(curs.fetchall())
 
result

,0
0,County
1,National
2,Query_results
3,State


The connection is successful and the expected table names were printed. Let's look at the basic information for National, State, and County tables:

In [6]:
curs.execute("SHOW columns FROM National")
national_headers = [column[0] for column in curs.fetchall()]

curs.execute("SELECT * FROM National")
national = pd.DataFrame(curs.fetchall(), columns=national_headers)

national.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1364 entries, 0 to 1363
Data columns (total 59 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              1364 non-null   int64  
 1   period_begin                    1364 non-null   object 
 2   period_end                      1364 non-null   object 
 3   period_duration                 1364 non-null   int64  
 4   region_type                     1364 non-null   object 
 5   region_type_id                  1364 non-null   int64  
 6   table_id                        1364 non-null   int64  
 7   is_seasonally_adjusted          1364 non-null   object 
 8   region                          1364 non-null   object 
 9   city                            1364 non-null   object 
 10  state                           1364 non-null   object 
 11  state_code                      1364 non-null   object 
 12  property_type                   13

In [7]:
curs.execute("SHOW columns FROM State")
state_headers = [column[0] for column in curs.fetchall()]

curs.execute("SELECT * FROM State")
state = pd.DataFrame(curs.fetchall(), columns=state_headers)

state.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28739 entries, 0 to 28738
Data columns (total 59 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              28739 non-null  int64  
 1   period_begin                    28739 non-null  object 
 2   period_end                      28739 non-null  object 
 3   period_duration                 28739 non-null  int64  
 4   region_type                     28739 non-null  object 
 5   region_type_id                  28739 non-null  int64  
 6   table_id                        28739 non-null  int64  
 7   is_seasonally_adjusted          28739 non-null  object 
 8   region                          28739 non-null  object 
 9   city                            28739 non-null  object 
 10  state                           28739 non-null  object 
 11  state_code                      28739 non-null  object 
 12  property_type                   

In [8]:
curs.execute("SHOW columns FROM County")
county_headers = [column[0] for column in curs.fetchall()]

curs.execute("SELECT * FROM County")
county = pd.DataFrame(curs.fetchall(), columns=county_headers)

county.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 640957 entries, 0 to 640956
Data columns (total 59 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              640957 non-null  int64  
 1   period_begin                    640957 non-null  object 
 2   period_end                      640957 non-null  object 
 3   period_duration                 640957 non-null  int64  
 4   region_type                     640957 non-null  object 
 5   region_type_id                  640957 non-null  int64  
 6   table_id                        640957 non-null  int64  
 7   is_seasonally_adjusted          640957 non-null  object 
 8   region                          640957 non-null  object 
 9   city                            640957 non-null  object 
 10  state                           640957 non-null  object 
 11  state_code                      640957 non-null  object 
 12  property_type   

We do not have any null values for areas of concern when loading the data. We can also try looking at some of the query results as follows:

In [11]:
curs.execute("SHOW columns FROM Query_results")
result2_headers = [column[0] for column in curs.fetchall()]

curs.execute("SELECT * FROM Query_results;")
result2 = pd.DataFrame(curs.fetchall(), columns=result2_headers)

result2.head()

,query_id,table_id,query,result,result_alternative,comments
0,1,State,State with the highest median home sale overall,"Columbia, 606740.8064516129","Connecticut, 2920000","Result uses average median sales overall and finds the max average,\n result alternative uses the max median without any averaging"
1,2,State,State with the highest median home sale in 2022,"Hawaii, 845585","Hawaii, 1746100","Result uses average median sales overall and finds the max average,\n result alternative uses the max median without any averaging"
2,3,State,State with the lowest median home sale overall,"Ohio, 134618.38709677418","Oklahoma, 240900","Result uses average median sales overall and finds the min average,\n result alternative uses the min median without any averaging"
3,4,State,State with the lowest median home sale in 2022,"Oklahoma, 190350","Oklahoma, 240900","Result uses average median sales overall and finds the min average,\n result alternative uses the min median without any averaging"
4,5,State,State with the greatest inventory overall,"Florida, 47378.4484","Florida, 150738","Result uses average inventory overall for each state and finds the max average,\n result alternative uses the max overall without any averaging"


This confirms that data from the MySQL database can be retrieved correctly.